In [29]:
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn import tree, preprocessing

In [2]:
df = pd.read_csv('../data_sets/students.csv')

In [3]:
df.columns = df.columns.map(lambda x: x.lower().replace(' ', '_'))

In [ ]:
df.loc[9]['age']

In [ ]:
df[df.army == 'могут призвать'].hair_length.mean()

In [ ]:
df.columns

In [ ]:
df[df.friend_number == 0].sex

In [ ]:
dog_lovers = df[df.animal == 'Собак']

In [ ]:
dog_lovers[dog_lovers.floor_number == 1].floor_number.size / dog_lovers.floor_number.size

In [ ]:
df[df['brother-sister'].str.contains('есть')].social_network_duration_min.median()

In [ ]:
df[df['brother-sister'].str.contains('нет')].social_network_duration_min.median()

In [ ]:
df_cut = df[['growth', 'weight', 'hair_length', 'year_of_birth', 'shoe_size']]

In [ ]:
df_cut.sort_values(by=['shoe_size', 'growth'], ascending=[False, True]).head(2)

In [ ]:
(df.growth + df.hair_length).max()

In [ ]:
df_cut.corr()

In [ ]:
df.groupby(['rock_paper_scissors'])[['russian_rating', 'maths_rating']].mean()

In [ ]:
df.describe()

In [ ]:
df.children_number.info()

In [ ]:
df[df.sex== 'женский'].weight.mean(), df[df.sex== 'мужской'].weight.mean()

In [ ]:
females = df[df.sex== 'женский'].weight.fillna(df[df.sex== 'женский'].weight.mean())

In [ ]:
males = df[df.sex== 'мужской'].weight.fillna(df[df.sex== 'мужской'].weight.mean())

In [ ]:
females, males

In [ ]:
females.append(males).mean()

In [ ]:
sublings = df[df['brother-sister'].str.contains('есть')]
singles = df[df['brother-sister'].str.contains('нет')]

In [ ]:
tmp = singles.children_number.fillna(singles.children_number.median())

In [ ]:
tmp2 = sublings.children_number.fillna(sublings.children_number.median())

In [ ]:
tmp.append(tmp2).describe()

In [ ]:
sns.set_theme(style="ticks", palette="pastel")
sns.boxplot(x='putin_age', data=df, palette=['m'])

In [ ]:
quan25 = df.putin_age.quantile(.25)
quan75 = df.putin_age.quantile(.75)
quan_dif = 1.5*(quan75 - quan25)

In [ ]:
normalize_putin_age = df[df.putin_age.between(quan25 - quan_dif, quan75 + quan_dif)]

In [ ]:
sns.set_theme(style="ticks", palette="pastel")
sns.boxplot(x='putin_age', data=normalize_putin_age, palette=['m'])

In [ ]:
normalize_putin_age.putin_age.describe()

In [ ]:
full_table = df.dropna()[['shoe_size', 'weight']]
kmeans_model = KMeans(n_clusters=2, random_state=1, n_init=10).fit(full_table)
full_table['labels'] = kmeans_model.labels_

In [ ]:
sns.scatterplot(full_table, y=full_table.shoe_size, x=full_table.weight, hue=full_table.labels)

In [ ]:
(full_table.labels == 0).count()

In [ ]:
df.age.values.reshape(-1,1)

In [ ]:
# 85 - yellow
# 15 - white

# 80 - ok
# 20 - mistake

In [ ]:
part_frame = df[['age', 'year_of_birth', 'month_of_birthday']]

In [ ]:
reg = LinearRegression()\
.fit(part_frame[['year_of_birth','month_of_birthday']].values.reshape(-1,2), y=part_frame.age.values)

In [ ]:
part_frame['predict'] = reg.predict(part_frame[['year_of_birth','month_of_birthday']].values.reshape(-1,2))

In [ ]:
part_frame

In [ ]:
mean_absolute_error(part_frame['age'], part_frame['predict'])

In [ ]:
test_students = pd.read_csv('../data_sets/students_test.csv')


In [ ]:
reg.predict(test_students[['year_of_birth','month_of_birthday']].values.reshape(-1,2))

# вероятность

In [35]:
full_df = pd.read_csv('../data_sets/students.csv')
full_df_test = pd.read_csv('../data_sets/students_test.csv')

full_df.columns = full_df.columns.map(lambda x: x.lower().replace(' ', '_'))
full_df_test.columns = full_df_test.columns.map(lambda x: x.lower().replace(' ', ''))

In [8]:
df=full_df[['growth','weight','sex']]
test_df=full_df_test[['growth','weight','sex']]

df = df.dropna()
test_df = test_df.dropna()


In [9]:
forest_model = RandomForestClassifier(max_depth=2, random_state=0)
forest_model.fit(df[['growth','weight']].values.reshape(-1,2), y=df.sex.values)

RandomForestClassifier(max_depth=2, random_state=0)

In [22]:
test_df['predict'] = forest_model.predict(test_df[['growth','weight']].values.reshape(-1,2))

probability = forest_model.predict_proba(test_df[['growth','weight']].values.reshape(-1,2))
test_df['prob_woman'] = probability[:, 0]
test_df['prob_man'] = probability[:, 1]


In [27]:
test_df.sort_values(by='prob_woman', ascending=False)

,growth,weight,sex,predict,prob_man,prob_woman
61,162.0,49.0,женский,женский,0.056850,0.943150
79,162.0,49.0,женский,женский,0.056850,0.943150
17,159.0,49.0,женский,женский,0.056850,0.943150
52,160.0,50.0,женский,женский,0.056920,0.943080
39,157.0,49.0,женский,женский,0.058491,0.941509
...,...,...,...,...,...,...
57,186.0,100.0,мужской,мужской,0.914950,0.085050
33,196.0,97.0,мужской,мужской,0.914950,0.085050
28,193.0,105.0,мужской,мужской,0.914950,0.085050
23,194.0,97.0,мужской,мужской,0.914950,0.085050


# Ищем институт. Какие признаки важны для этого 

In [63]:
coder = preprocessing.LabelEncoder()

In [78]:
categorial_columns = ['sex', 'coin', 'animal', 'army', 'glasses', 'your_rating_in_university', 'fastfood',
                     'hostel', 'chocolate', 'brother-sister', 'plane_seat', 'problems_in_last_semester', 
                      'rock_paper_scissors', 'strange_people', 'your_insitute'
                     ]
#object_col = [col for col in df.columns if df[col].dtype==object]

In [79]:
# Цифровизуем признаки
cat_df = full_df
cat_df = cat_df.dropna()
for name in categorial_columns:
    coder.fit(cat_df[name])
    cat_df[name] = coder.transform(cat_df[name])

/tmp/ipykernel_12932/3441455394.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_df[name] = coder.transform(cat_df[name])
/tmp/ipykernel_12932/3441455394.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_df[name] = coder.transform(cat_df[name])
/tmp/ipykernel_12932/3441455394.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [80]:
selector = ExtraTreesClassifier()
result = selector.fit(cat_df, cat_df.your_insitute)
result.feature_importances_

array([0.01379778, 0.01410005, 0.01889325, 0.00845003, 0.00918627,
       0.01683784, 0.01612761, 0.06209791, 0.01156979, 0.08050101,
       0.00907296, 0.02658442, 0.04907821, 0.01167486, 0.01403013,
       0.02305154, 0.06403327, 0.0136191 , 0.01375285, 0.01756961,
       0.00908666, 0.01192926, 0.01340641, 0.01713737, 0.00637692,
       0.01328429, 0.01130246, 0.008858  , 0.01061926, 0.00692501,
       0.01125756, 0.0150213 , 0.01472603, 0.01127241, 0.00550252,
       0.00439049, 0.01304118, 0.01065977, 0.01444016, 0.01952191,
       0.00845459, 0.01492578, 0.20285181, 0.00763841, 0.01065694,
       0.01212147, 0.01062842, 0.00993513])

In [81]:
features_table = pd.DataFrame(result.feature_importances_, index=cat_df.columns, columns=['importance'])

In [82]:
features_table

,importance
age,0.013798
growth,0.014100
shoe_size,0.018893
course_number,0.008450
year_of_birth,0.009186
friend_number,0.016838
russian_rating,0.016128
maths_rating,0.062098
physics_rating,0.011570
computer_science_rating,0.080501
